In [1]:
# Importation des librairies
from my_function import ( 
    train_random_forest, 
    save_classification,
    sel_classif_pixel,
    supprimer_dossier_non_vide)
import os
import geopandas as gpd
import sys
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw
# import classification as cla

In [2]:
# Création du dossier de sauvegarde temporaire
output_dir = "/home/onyxia/work/output"
os.makedirs(output_dir, exist_ok=True) # Création d'un dossier output temporaire

In [8]:
sptial_resolution = rw.get_image_dimension(ref_image_open)[0]
xmin,ymin = rw.get_origin_coordinates(ref_image_open)
y,x = rw.get_image_dimension(ref_image_open)[0:2]
xmax,ymax = xmin+x,ymin+y
print(xmin,ymin,xmax,ymax)

501127.9696999999 6314464.023599998 511990.9696999999 6321845.023599998


In [9]:
gpd.read_file(in_vector).head()

,ID,CODE_TFV,TFV,TFV_G11,ESSENCE,Nom,Code,geometry
0,FORESTIE0000000003108720,FF1-00,Forêt fermée de feuillus purs en îlots,Forêt fermée feuillus,Feuillus,Feuillus en îlots,16,"POLYGON ((542016.264 6240744.823, 542019.845 6..."
1,FORESTIE0000000003108753,FF1-00,Forêt fermée de feuillus purs en îlots,Forêt fermée feuillus,Feuillus,Feuillus en îlots,16,"POLYGON ((542369.772 6240980.009, 542369.28 62..."
2,FORESTIE0000000003116029,FF1-00-00,Forêt fermée à mélange de feuillus,Forêt fermée feuillus,Feuillus,Mélange de feuillus,15,"POLYGON ((541667.758 6240990.232, 541662.797 6..."
3,FORESTIE0000000003108760,FF1-00,Forêt fermée de feuillus purs en îlots,Forêt fermée feuillus,Feuillus,Feuillus en îlots,16,"POLYGON ((545450.8 6241119, 545446 6241115.3, ..."
4,FORESTIE0000000003116048,FF1-00-00,Forêt fermée à mélange de feuillus,Forêt fermée feuillus,Feuillus,Mélange de feuillus,15,"POLYGON ((545762.816 6241493.748, 545772.766 6..."


In [4]:
# cmd rasterisation paramètres
in_vector = '/home/onyxia/work/Depot_Git/results/data/sample/Sample_BD_foret_T31TCJ.shp'
ref_image = '/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_allbands.tif'
ref_image_open = rw.open_image(ref_image)
out_image = os.path.join(output_dir, "rasterized_sample.tif")
field_name = 'Code'  # field containing the numeric label of the classes
sptial_resolution = rw.get_image_dimension(ref_image_open)[0]
xmin,ymin = rw.get_origin_coordinates(ref_image_open)
y,x = rw.get_image_dimension(ref_image_open)[0:2]
xmax,ymax = xmin+x,ymin+y

# define command pattern to fill with paremeters
cmd_pattern = ("gdal_rasterize -a {field_name} "
               "-tr {sptial_resolution} {sptial_resolution} "
               "-te {xmin} {ymin} {xmax} {ymax} -ot Byte -of GTiff "
               "{in_vector} {out_image}")

# fill the string with the parameter thanks to format function
cmd = cmd_pattern.format(in_vector=in_vector, xmin=xmin, ymin=ymin, xmax=xmax,
                         ymax=ymax, out_image=out_image, field_name=field_name,
                         sptial_resolution=sptial_resolution)

# execute the command in the terminal
os.system(cmd)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [7]:
test = rw.load_img_as_array('/home/onyxia/work/output/rasterized_sample.tif')
test

array([[[0]]], dtype=uint8)

In [ ]:
# Entraîner le modèle
model = train_random_forest(Samples_X, Class_Y)

In [ ]:
# Sauvegarder la carte
save_classification(model, images, 'results/data/classif/carte_essences_echelle_pixel.tif')

In [ ]:
## Nettoyage des dossiers
supprimer_dossier_non_vide(output_dir)